In [1]:
import pandas as pd

df = pd.read_csv('kc_house_data_test_features.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
print(df.shape)
df.head()

(4323, 20)


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


# Apply Changes from Train Data onto Holdout Set

### Cleaning Original Features

In [2]:
#cap sqft_above values at 7K sqft, re-look at scatter plot
import numpy as np

df['sqft_above'] = np.where(df['sqft_above'] > 7000, 7000, df['sqft_above'])


### Creating New Features

In [3]:
#transform grade/bathrooms into e^grade/e^bathroom to make more linear
df['grade_exp'] = np.exp(df['grade'])
df['bath_exp'] = np.exp(df['bathrooms'])

#whether or not the house has a basement
df['basement'] = np.where(df['sqft_basement'] > 0, 1, 0)

#sqft of outdoor space
df['sqft_outdoor'] = df['sqft_lot'] - (df['sqft_living']/df['floors'])

#clean sqft_outdoor--cap at 600K
df['sqft_outdoor'] = np.where(df['sqft_outdoor'] > 600000, 600000, df['sqft_outdoor'])


In [4]:
#distance to amazon headquarters
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

amazon = (47.623583541669845, -122.33669143795257)

dist_amazon = []
for row in df.itertuples(index=False):
    dist_amazon.append(haversine_distance(row.lat, row.long, amazon[0], amazon[1]))

df['dist_amazon'] = dist_amazon

#distance to microsoft headquarters

microsoft = (47.685228949452586, -122.09372667339042)

dist_microsoft = []
for row in df.itertuples(index=False):
    dist_microsoft.append(haversine_distance(row.lat, row.long, microsoft[0], microsoft[1]))

df['dist_microsoft'] = dist_microsoft



In [5]:
#feature for Seattle neighborhood (or 'None' if not in Seattle)
df['seattle_neighborhood'] = 'None'
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98133|98177, 'Northwest', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98101|98104|98111|98114|98121|98129|98154|98161|98164|98174|98181|98184|98191, 'Downtown', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98102|98112, 'Capitol Hill', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98103|98103, 'Lake Union', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98105|98115|98145|98185|98195, 'Northeast', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98106|98106|98126 , 'Delridge', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98107|98117, 'Ballard', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98108|98124|98134, 'Duwamish', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98109|98119|98199, 'Queen Anne/Magnolia', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98116|98136|98146, 'Southwest', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98118|98144, 'Southeast', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98122, 'Central', df['seattle_neighborhood'])
df['seattle_neighborhood'] = np.where(df['zipcode'] == 98125, 'North', df['seattle_neighborhood'])

### Create dummy variables

In [12]:
df = pd.get_dummies(df, columns=['seattle_neighborhood', 'bedrooms'], drop_first=True)


### Apply Polynomial Funciton

In [18]:
df.columns


Index(['id', 'date', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'grade_exp', 'bath_exp', 'basement',
       'sqft_outdoor', 'dist_amazon', 'dist_microsoft',
       'seattle_neighborhood_Central', 'seattle_neighborhood_Lake Union',
       'seattle_neighborhood_None', 'seattle_neighborhood_North', 'bedrooms_1',
       'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5', 'bedrooms_6',
       'bedrooms_7', 'bedrooms_8', 'bedrooms_9', 'bedrooms_10'],
      dtype='object')

In [19]:
features = ['sqft_living', 'waterfront', 'sqft_living15',
            'basement', 'sqft_outdoor', 'dist_amazon', 'dist_microsoft', 'grade_exp', 'bath_exp',
            'seattle_neighborhood_Central', 'seattle_neighborhood_Lake Union', 'seattle_neighborhood_None',
            'seattle_neighborhood_North', 'bedrooms_1', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4',
            'bedrooms_5', 'bedrooms_6', 'bedrooms_7','bedrooms_8','bedrooms_9','bedrooms_10']

In [20]:
df[features]


,sqft_living,waterfront,sqft_living15,basement,sqft_outdoor,dist_amazon,dist_microsoft,grade_exp,bath_exp,seattle_neighborhood_Central,...,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bedrooms_7,bedrooms_8,bedrooms_9,bedrooms_10
0,2270,0,2020,1,9230.0,11.89,11.33,2980.957987,12.182494,0,...,0,0,0,1,0,0,0,0,0,0
1,2270,0,2020,1,9230.0,11.89,11.33,2980.957987,12.182494,0,...,0,0,0,1,0,0,0,0,0,0
2,1470,0,1470,1,1044.0,26.78,16.94,2980.957987,12.182494,0,...,0,0,1,0,0,0,0,0,0,0
3,1280,0,1160,1,14920.0,23.90,6.61,2980.957987,5.754603,0,...,0,0,1,0,0,0,0,0,0,0
4,2830,0,2830,0,6711.0,21.23,22.39,2980.957987,15.642632,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,1530,0,1530,0,621.0,8.45,18.95,2980.957987,12.182494,0,...,0,0,1,0,0,0,0,0,0,0
4319,2310,0,1830,0,4658.0,12.69,27.95,2980.957987,12.182494,0,...,0,0,0,1,0,0,0,0,0,0
4320,1020,0,1020,0,840.0,4.30,18.40,1096.633158,2.117000,0,...,0,1,0,0,0,0,0,0,0,0
4321,1600,0,1410,0,1588.0,22.39,16.86,2980.957987,12.182494,0,...,0,0,1,0,0,0,0,0,0,0


In [21]:
#add polnomial features
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)

poly_data = poly.fit_transform(df[features])

poly_columns = poly.get_feature_names(features)

df_poly = pd.DataFrame(poly_data, columns=poly_columns)

df_poly.head()

,sqft_living,waterfront,sqft_living15,basement,sqft_outdoor,dist_amazon,dist_microsoft,grade_exp,bath_exp,seattle_neighborhood_Central,...,bedrooms_7^2,bedrooms_7 bedrooms_8,bedrooms_7 bedrooms_9,bedrooms_7 bedrooms_10,bedrooms_8^2,bedrooms_8 bedrooms_9,bedrooms_8 bedrooms_10,bedrooms_9^2,bedrooms_9 bedrooms_10,bedrooms_10^2
0,2270.0,0.0,2020.0,1.0,9230.0,11.89,11.33,2980.957987,12.182494,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2270.0,0.0,2020.0,1.0,9230.0,11.89,11.33,2980.957987,12.182494,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1470.0,0.0,1470.0,1.0,1044.0,26.78,16.94,2980.957987,12.182494,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1280.0,0.0,1160.0,1.0,14920.0,23.90,6.61,2980.957987,5.754603,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2830.0,0.0,2830.0,0.0,6711.0,21.23,22.39,2980.957987,15.642632,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Define Selected Columns

In [22]:
selected_columns = ['sqft_living','sqft_living15','dist_microsoft','grade_exp','bath_exp',
                    'sqft_living^2','sqft_living waterfront','sqft_living sqft_living15','sqft_living basement',
                    'sqft_living grade_exp','sqft_living bath_exp','sqft_living seattle_neighborhood_None','sqft_living15^2',
                    'sqft_living15 basement','sqft_living15 grade_exp','sqft_living15 bath_exp','sqft_living15 seattle_neighborhood_None',
                    'basement grade_exp','basement bath_exp','dist_amazon dist_microsoft','dist_amazon grade_exp',
                    'dist_microsoft^2','dist_microsoft grade_exp','dist_microsoft bath_exp','dist_microsoft seattle_neighborhood_None','grade_exp^2','grade_exp seattle_neighborhood_None','grade_exp bedrooms_4','bath_exp seattle_neighborhood_None','bath_exp bedrooms_4']

# Apply Scaler

In [23]:
import pickle

scaler = pd.read_pickle('scaler.pickle')
model = pd.read_pickle('model.pickle')


In [26]:
from sklearn.preprocessing import StandardScaler

transformed_holdout = scaler.transform(df_poly[selected_columns])


In [27]:
answers = model.predict(transformed_holdout)


In [33]:
pd.DataFrame(answers).to_csv('housing_preds_DavidaRosenstrauch.csv')

# answers.to_csv('housing_preds_DavidaRosenstrauch.csv')
